# sqlite

`sqlite3` package implements a sqlite database in the python. It's usually build in package. Check [its page](https://docs.python.org/3/library/sqlite3.html) in python.org.

In [ ]:
import sqlite3

## Usage

To use database you need: create connection, get cursor, manipulate with database using cursor methods.

---

The following cell creates the connection - it passes `:memory:` to use the database only in memory without any file where data will be stored.

In [22]:
con = sqlite3.connect(':memory:')

Next code shows how to create cursor and load data from it.

In [23]:
cursor = con.cursor()
cursor.execute("SELECT 1 as val1, 2 as val2;")

The following cells show how you can extract data and ged description to it.

In [24]:
cursor.fetchall()

[(1, 2)]

In [25]:
cursor.description

(('val1', None, None, None, None, None, None),
 ('val2', None, None, None, None, None, None))